In [1]:
import pandas as pd
import numpy as np

In [22]:
import h2o

In [152]:
train_data = pd.read_csv("C:/Users/Leonid/Documents/git/lkulighin/bnp/train.csv")
Y_train=train_data['target']
X_train=train_data[[c for c in train_data.columns if c not in ('ID', 'target')]]
test_data = pd.read_csv("C:/Users/Leonid/Documents/git/lkulighin/bnp/test.csv")
X_test=test_data[[c for c in test_data.columns if c not in ('ID', 'target')]]

In [104]:
def find_corr(dt, rate):
    i = 0
    types = dt.dtypes.to_dict()
    for c1 in dt.columns[:-1]:
        i += 1
        for c2 in dt.columns[i:]:
            if (types[c1] in ('float64', 'int64')) & (types[c2] in ('float64', 'int64')):
                v1 = dt[dt[c1].notnull() & dt[c2].notnull()][c1]
                v2 = dt[dt[c1].notnull() & dt[c2].notnull()][c2]
                cr = np.corrcoef(v1,v2)[0][1]
                if abs(cr) > rate:
                    print "{0} and {1} are correlated: {2:0.2f}".format(c1, c2, cr)
    return

In [122]:
def binarize(train_data, c, **kwargs):
    train = train_data.apply(lambda x: str(x))
    lbls = train.unique()
    dct_train = {}
    dct_test = {}
    l_train = train_data.shape[0]
    l_test = -1
    if 'test_data' in kwargs:
        test = kwargs['test_data'].apply(lambda x: str(x))
        l_test = test.shape[0]
    for x in lbls:
        dct_train[x]=np.zeros(l_train)
        if l_test > 0:
            dct_test[x]=np.zeros(l_test)
    res_train = pd.DataFrame(dct_train)
    for i in range(l_train):
        res_train[train.iloc[i]].iloc[i] = 1
    res_train.columns = [c + '-' + x for x in res_train.columns]
    if l_test > 0:
        res_test = pd.DataFrame(dct_test)
        for i in range(l_test):
            if test.iloc[i] in lbls:
                res_test[test.iloc[i]].iloc[i] = 1
        res_test.columns = [c + '-' + x for x in res_test.columns]
        return res_train[res_train.columns[:-1]], res_test[res_test.columns[:-1]]
    return res_train[res_train.columns[:-1]]

In [ ]:
print find_corr(data, 0.85)

In [130]:
print [(c, len(data[c].unique()), data[c].dtype) for c in data.columns if data[c].dtypes not in ('float64', 'int64')]

[('v3', 4, dtype('O')), ('v22', 18211, dtype('O')), ('v24', 5, dtype('O')), ('v30', 8, dtype('O')), ('v31', 4, dtype('O')), ('v47', 10, dtype('O')), ('v52', 13, dtype('O')), ('v56', 123, dtype('O')), ('v66', 3, dtype('O')), ('v71', 9, dtype('O')), ('v74', 3, dtype('O')), ('v75', 4, dtype('O')), ('v79', 18, dtype('O')), ('v91', 8, dtype('O')), ('v107', 8, dtype('O')), ('v110', 3, dtype('O')), ('v112', 23, dtype('O')), ('v113', 37, dtype('O')), ('v125', 91, dtype('O'))]


In [46]:
print len(data['v3'].unique())

4


In [151]:
print test_data.columns

Index([u'ID', u'v1', u'v2', u'v3', u'v4', u'v5', u'v6', u'v7', u'v8', u'v9', 
       ...
       u'v122', u'v123', u'v124', u'v125', u'v126', u'v127', u'v128', u'v129',
       u'v130', u'v131'],
      dtype='object', length=132)


In [156]:
for c in X_train.columns:
    if train_data[c].dtype == 'O':
        print c
        tmp1, tmp2 = binarize(X_train[c], c, test_data = test_data[c])
        X_train.drop(c, axis = 1)
        X_test.drop(c, axis = 1)
        X_train = pd.concat([X_train, tmp1], axis = 1)
        X_test = pd.concat([X_test, tmp2], axis = 1)

v3
v22


MemoryError: 

In [166]:
print X_train[X_train['v1'].isnull()]['v1']

Series([], Name: v1, dtype: float64)


In [172]:
print test_data['ID'].values

[     0      1      2 ..., 228704 228706 228709]


In [174]:
res = pd.DataFrame({'ID': test_data['ID'].values})

In [175]:
print res

            ID
0            0
1            1
2            2
3            7
4           10
5           11
6           13
7           14
8           15
9           16
10          17
11          18
12          19
13          20
14          25
15          26
16          29
17          38
18          41
19          44
20          45
21          47
22          48
23          49
24          50
25          53
26          56
27          59
28          60
29          64
...        ...
114363  228650
114364  228656
114365  228661
114366  228662
114367  228664
114368  228665
114369  228667
114370  228669
114371  228672
114372  228674
114373  228675
114374  228676
114375  228678
114376  228679
114377  228681
114378  228685
114379  228686
114380  228687
114381  228689
114382  228690
114383  228692
114384  228693
114385  228694
114386  228696
114387  228698
114388  228700
114389  228703
114390  228704
114391  228706
114392  228709

[114393 rows x 1 columns]


In [ ]:
for c in X_train.columns:
    if train_data[c].dtype <> 'O':
        print c, len()

In [ ]:
print X_train.columns

In [125]:
x = pd.Series(['A', 'A', np.nan, 'B', 3.14, np.inf])
y = pd.Series(['B', 'A', np.nan, 'D'])

In [126]:
a, b = binarize(x, 'v1', test_data = y)
print a
print b

   v1-3.14  v1-A  v1-B  v1-inf
0        0     1     0       0
1        0     1     0       0
2        0     0     0       0
3        0     0     1       0
4        1     0     0       0
5        0     0     0       1
   v1-3.14  v1-A  v1-B  v1-inf
0        0     0     1       0
1        0     1     0       0
2        0     0     0       0
3        0     0     0       0
